In [1]:
!pip install psycopg2-binary
!pip install pandas

     |████████████████████████████████| 3.4 MB 109 kB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 185 kB/s eta 0:00:01
     |████████████████████████████████| 15.7 MB 1.6 MB/s eta 0:00:01


In [1]:
import psycopg2
import pandas as pd
import db_engine as dbe

In [2]:
params = {
    "host"      : "pg_container",
    "database"  : "cargill_db",
    "user"      : "cargill",
    "password"  : "cargill"
}

In [3]:
cur, conn = dbe.create_connection(params)

Connecting to the PostgreSQL database
PostgreSQL database version:
('PostgreSQL 13.3 (Debian 13.3-1.pgdg100+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 8.3.0-6) 8.3.0, 64-bit',)


In [4]:
dbe.drop_tables(cur, conn)

Executing: DROP TABLE IF EXISTS Sales
Executing: DROP TABLE IF EXISTS Customers
Executing: DROP TABLE IF EXISTS Products
Executing: DROP TABLE IF EXISTS Suppliers
Executing: DROP TABLE IF EXISTS Discounts


In [5]:
dbe.create_tables(cur, conn)

In [6]:
sales_insert = ("""
insert into sales values (1, 2, 150, NOW(), 200, 200),(1, 3, 150, NOW(), 310, 310),(1, 4, 150, NOW(), 80, 80)
""")

cur.execute(sales_insert)
conn.commit() 

discounts_insert = ("""
insert into discounts values (1, 150, 0.3)
""")

cur.execute(discounts_insert)
conn.commit() 

In [8]:
columns = ["sales_order_id", "sales_order_item", "customer_id", "date", "transaction_value", "discounted_value" ]
df = dbe.pg_to_pd(cur, "select * from sales", columns)
df.head()

,sales_order_id,sales_order_item,customer_id,date,transaction_value,discounted_value
0,1,2,150,2021-07-15 15:39:42.456341,200.0,200.0
1,1,3,150,2021-07-15 15:39:42.456341,310.0,310.0
2,1,4,150,2021-07-15 15:39:42.456341,80.0,80.0


In [9]:
columns = ["sales_order_id", "customer_id", "discount_value" ]
df = dbe.pg_to_pd(cur, "select * from discounts", columns)
df.head()

,sales_order_id,customer_id,discount_value
0,1,150,0.3


In [33]:
sales_update = (
"""

WITH sales_updated_CTE(sales_order_id,sales_order_item, customer_id, date,transaction_value, discount)
AS
(
    SELECT 
        s.sales_order_id,
        s.sales_order_item,
        s.customer_id,
        s.date,
        s.transaction_value,
        (s.transaction_value - (s.transaction_value * d.discount_value)) AS discount
    FROM sales s
    JOIN discounts d
    ON s.sales_order_id = d.sales_order_id
    WHERE s.sales_order_id = %s AND s.customer_id = %s
)
UPDATE sales as s
SET discounted_value = c.discount
FROM sales_updated_CTE as c 
WHERE c.sales_order_id = s.sales_order_id 
AND c.customer_id = s.customer_id
AND c.sales_order_item = s.sales_order_item
"""
)

In [34]:
cur.execute(sales_update, [1, 150])
conn.commit() 

In [35]:
columns = ["sales_order_id", "sales_order_item", "customer_id", "date", "transaction_value", "discounted_value" ]
df = dbe.pg_to_pd(cur, "select * from sales", columns)
df.head()

,sales_order_id,sales_order_item,customer_id,date,transaction_value,discounted_value
0,1,2,150,2021-07-15 15:39:42.456341,200.0,140.0
1,1,3,150,2021-07-15 15:39:42.456341,310.0,217.0
2,1,4,150,2021-07-15 15:39:42.456341,80.0,56.0
